In [21]:
!pip3 install pyomo
import pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached glpk-0.4.7.tar.gz (161 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: Could not build wheels for glpk, which is required to install pyproject.toml-based projects


In [84]:

    B = 9
    w = [2,3,4,5,6,7,8]
    q = [4,2,6,6,2,2,2]
    s=[]
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])

In [85]:
def FFD(s, B):
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for j,free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol
n = len(s)
U = len(FFD(s, B))

In [86]:

    model = ConcreteModel()

    # Variables
    model.x = Var(range(n), range(U), within=Binary, initialize=0)
    model.y = Var(range(U), within=Binary, initialize=0)

    # Constraints
    # Each item must be in exactly one bin
    def assignment_constraint(model, i):
        return sum(model.x[i, j] for j in range(U)) == 1
    model.assignment_constraint = Constraint(range(n), rule=assignment_constraint)

    # The total weight in each bin cannot exceed the bin capacity times the bin use variable
    def capacity_constraint(model, j):
        return sum(s[i] * model.x[i, j] for i in range(n)) <= B * model.y[j]
    model.capacity_constraint = Constraint(range(U), rule=capacity_constraint)

    # x_ij can only be 1 if y_j is 1 (i.e., if bin j is used)
    def linking_constraint(model, i, j):
        return model.x[i, j] <= model.y[j]
    model.linking_constraint = Constraint(range(n), range(U), rule=linking_constraint)

    # Objective
    model.obj = Objective(expr=sum(model.y[j] for j in range(U)), sense=minimize)



In [87]:

    solver = SolverFactory('glpk')
    solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 13.0, 'Upper bound': 13.0, 'Number of objectives': 1, 'Number of constraints': 349, 'Number of variables': 325, 'Number of nonzeros': 1261, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '47', 'Number of created subproblems': '47'}}, 'Error rc': 0, 'Time': 0.03950691223144531}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [88]:
print("Objective Value:", model.obj())

Objective Value: 13.0


In [92]:
bins = [[] for i in range(U)]
for (i,j) in model.x:
  if value(model.x[i,j])> .5:
    bins[j].append(s[i])
for i in range(bins.count([])):
  bins.remove([])
for b in bins:
  b.sort()
bins.sort()

In [94]:
print("Bin Division:", bins)

Bin Division: [[2, 2, 5], [2, 7], [2, 7], [3, 6], [3, 6], [4, 4], [4, 5], [4, 5], [4, 5], [4, 5], [5], [8], [8]]
